In [2]:
import networkx as nx
%matplotlib qt 
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import src.resistor as resistor 
from importlib import reload

# Check 1D network

In [3]:
reload(resistor)
ntwrk = resistor.Network()
noOfNodes = 10
for i in range(noOfNodes-1):
    ntwrk.add_edge(i,i+1,1)
print(ntwrk.edges)
print(ntwrk.nodes)
for i in range(1,noOfNodes-1):
    ntwrk.star_mesh(i)
print(ntwrk.edges)
print(ntwrk.nodes)


{(1, 0): 1, (2, 1): 1, (3, 2): 1, (4, 3): 1, (5, 4): 1, (6, 5): 1, (7, 6): 1, (8, 7): 1, (9, 8): 1}
{0: {1: 1}, 1: {0: 1, 2: 1}, 2: {1: 1, 3: 1}, 3: {2: 1, 4: 1}, 4: {3: 1, 5: 1}, 5: {4: 1, 6: 1}, 6: {5: 1, 7: 1}, 7: {6: 1, 8: 1}, 8: {7: 1, 9: 1}, 9: {8: 1}}
{(9, 0): 9.0}
{0: {9: 1}, 9: {0: 1}}


# Initialize 2D network

In [4]:
%%time
reload(resistor)
def xfy(x,l,rad=0.3):
    d = rad*l
    R = (l**2 + d**2)/(2*d)
    y = (R**2 - (l-x)**2)**0.5 - R + d
    return y
def init_sample_network(
    tViz = 0.001,
    Xlen = 10,
    Ylen = 10):
    global fig,ax,start_node,end_node,psG,ntwrk,lx,ly
    start_node = (Xlen//2,0)
    end_node = (Xlen//2,Ylen-1)
    ntwrk = resistor.Network()
    for i in range(Xlen):
        for j in range(Ylen):
            if j+1<Ylen:
                ntwrk.add_edge((i,j),(i,(j+1)),1)
            if i+1<Xlen:
                if j==0 or j==Ylen-1:
                    ntwrk.add_edge((i,j),((i+1),j),0)
                else:
                    ntwrk.add_edge((i,j),((i+1),j),1)
    # print(ntwrk.edges)
    # print(ntwrk.nodes[Xlen*Ylen-1])
    fig,ax = plt.subplots(figsize=(10,10))
    plt.ion()
    lx = Xlen-1
    ly = Ylen-1
    psG = {k:(k[0]+xfy(k[1],ly/2),k[1]+xfy(k[0],lx/2)) for k in ntwrk.nodes.keys()}
    ax.axis('off')
    nx.draw_networkx(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
    weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
    nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
    fig.canvas.draw()
    plt.pause(tViz)
    return tViz,Xlen,Ylen

Wall time: 1.98 ms


# Brute force with star-mesh transform

In [5]:
%%time
reload(resistor)
tViz,Xlen,Ylen = init_sample_network()

Wall time: 1.39 s


In [9]:
for i in np.random.permutation(Xlen):
    for j in np.random.permutation(Ylen):
        if (i,j) not in [start_node,end_node]:
            ntwrk.star_mesh((i,j))
            print(f"\rRemoved node:({i},{j}) ... ",end='',flush=True)
            ax.cla()
            ax.axis('off')
            nx.draw_networkx(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
            # weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
            # nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
            fig.canvas.draw()
            plt.pause(tViz)
print("Done!")
print(ntwrk.edges)
print(ntwrk.nodes)

Removed node:(7,6) ... Done!
{((5, 9), (5, 0)): 0.9}
{(5, 0): {(5, 9): 1}, (5, 9): {(5, 0): 1}}


In [11]:
%%time
reload(resistor)
tViz,Xlen,Ylen = init_sample_network()

Wall time: 1.53 s


In [13]:
for i in range(Xlen):
    for j in range(Ylen):
        if (i,j) not in [start_node,end_node]:
            ntwrk.star_mesh((i,j))
            print(f"\rRemoved node:({i},{j}) ... ",end='',flush=True)
            ax.cla()
            # ax.axis('off')
            nx.draw(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
            fig.canvas.draw()
            plt.pause(tViz)
weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
print("Done!")
print(ntwrk.edges)
print(ntwrk.nodes)

Removed node:(9,9) ... Done!
{((5, 9), (5, 0)): 0.8999999999999998}
{(5, 0): {(5, 9): 1}, (5, 9): {(5, 0): 1}}


# Bond propagation algorithm

## Construct graph

In [25]:
%%time
reload(resistor)
tViz,Xlen,Ylen = init_sample_network()

Wall time: 935 ms


## Connect shorted edges

In [26]:
%%time
tViz = 0.001
while 0 in ntwrk.edges.values():
    edgey = list(ntwrk.edges.values()).index(0)
    edge = list(ntwrk.edges.keys())[edgey]
    if edge[0] not in [start_node,end_node]:
        if edge[1] not in [start_node,end_node]:
            node = min(edge[0],edge[1])
        else:
            node = edge[1]
    else:
        if edge[1] not in [start_node,end_node]:
            node = edge[0]
        else:
            continue
    iShort = ntwrk.merge_short(edge,node)
    if iShort:
        print(f"\rRemoved edge:{edge} ... ",end='',flush=True)
        ax.cla()
        ax.axis('off')
        nx.draw(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
        weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
        nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
        fig.canvas.draw()
        plt.pause(tViz)


Removed edge:((9, 9), (5, 9)) ... Wall time: 11.4 s


## Propagate bonds

In [27]:
def propagate(node1,node2,node3,update_plot=True):
    node = ntwrk.delta_wye(node1,node2,node3)
    if update_plot:
        psG[node] = (node[0]+xfy(node[1],ly/2),node[1]+xfy(node[0],lx/2))
        ax.cla()
        nx.draw(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
        weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
        nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
        fig.canvas.draw()
        plt.pause(tViz)
    ntwrk.star_mesh(node3)
    if update_plot:
        ax.cla()
        nx.draw(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
        weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
        nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
        fig.canvas.draw()
        plt.pause(tViz)
    ntwrk.shift_node(node,node3)
    if update_plot:
        ax.cla()
        nx.draw(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
        weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
        nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
        fig.canvas.draw()
        plt.pause(tViz)

for j in range(1,Ylen-1):
    for i in range(Xlen):
        if (i,j) not in [start_node,end_node]:
            ntwrk.star_mesh((i,j))
            ax.cla()
            nx.draw(ntwrk.nxG, pos=psG, node_size=0, ax=ax, connectionstyle="arc3, rad=0.2")
            weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
            nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
            fig.canvas.draw()
            plt.pause(tViz)
            ii=i
            jj=j
            while ii+1<Xlen and jj+1<Ylen-1:
                print(f"\rPropagating edge:({ii},{jj+1}),({ii+1},{jj}) ... ",end='',flush=True)
                propagate((ii,jj+1),(ii+1,jj),(ii+1,jj+1))
                ii+=1
                jj+=1
weights = nx.get_edge_attributes(ntwrk.nxG,'weight')
nx.draw_networkx_edge_labels(ntwrk.nxG, pos=psG, ax=ax, edge_labels=weights)
print("Done!")
print(ntwrk.edges)
print(ntwrk.nodes)

Propagating edge:(6,5) ... 

KeyboardInterrupt: 

Propagating edge:(8,6) ... 

KeyboardInterrupt: 

In [33]:
type(1)

int

---
# The End